# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
#Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

#Import Google Key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Load CSV file as DataFrame
city_data = pd.read_csv("../WeatherPy/Data/City_Data.csv")
city_data.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Date,Max Temp,Humidity,Clouds,Wind Speed
0,0,Punta Arenas,CL,-53.15,-70.92,1601716955,41.00,65,75,19.46
1,1,Bonthe,SL,7.53,-12.51,1601716956,78.48,86,99,3.06
2,2,Kapaa,US,22.08,-159.32,1601716957,80.60,83,75,4.70
3,3,Nanortalik,GL,60.14,-45.24,1601716958,37.04,91,97,12.97
4,4,Naze,JP,28.37,129.48,1601716959,80.60,69,20,11.41
5,5,Half Moon Bay,US,37.46,-122.43,1601716959,64.00,77,90,4.90
6,6,Entre Rios,BR,-11.94,-38.08,1601716960,68.05,93,16,1.45
7,7,George,ZA,-33.96,22.46,1601716960,60.80,93,75,18.34
8,8,Mossamedes,AO,-15.20,12.15,1601716961,66.20,82,75,13.87
9,9,Darhan,MN,49.49,105.92,1601716962,40.33,57,2,6.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Set Humidity as Weight
humidity = city_data["Humidity"].astype(float)
maxhumidity = humidity.max()

#Set Lat and Lng as Locations
locations = city_data[["Lat", "Lng"]]

In [6]:
#Plot Map
fig = gmaps.figure()

#Add Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)

#Show Map
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#Save Map as PNG
plt.savefig("Images/Heat Map.png")

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Narrow Cities to Reflect Conditions
narrowed_cities = city_data.loc[(city_data["Max Temp"] > 70) & (city_data["Max Temp"] < 80) & (city_data["Clouds"] == 0), :]

#Drop Null Values
narrowed_cities = narrowed_cities.dropna(how='any')
narrowed_cities.reset_index(inplace=True)

#Delete Unneccessary Columns
del narrowed_cities["Unnamed: 0"]
del narrowed_cities["index"]

#Show DataFrame
narrowed_cities.head(10)

,City,Country,Lat,Lng,Date,Max Temp,Humidity,Clouds,Wind Speed
0,Seydi,TM,39.48,62.91,1601716965,75.87,17,0,5.19
1,Atar,MR,20.52,-13.05,1601717042,78.80,44,0,16.11
2,Cabo San Lucas,MX,22.89,-109.91,1601717023,78.01,75,0,1.99
3,Qarqīn,AF,37.42,66.04,1601717058,79.81,15,0,3.74
4,Akşehir,TR,38.36,31.42,1601717063,70.68,25,0,3.58
5,Guerrero Negro,MX,27.98,-114.06,1601717109,73.78,66,0,6.02
6,Coahuayana Viejo,MX,18.73,-103.68,1601717143,72.82,57,0,5.41
7,Erzin,TR,36.96,36.20,1601717193,79.00,67,0,5.66
8,São João da Barra,BR,-21.64,-41.05,1601717256,73.40,94,0,11.41
9,San Borja,BO,-14.82,-66.85,1601717263,75.43,63,0,1.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Create Empty List for Hotels
hotel_df = []

#Begin For Loop to Find Hotels
for i in range(len(narrowed_cities)):
    lat = narrowed_cities.loc[i]['Lat']
    lng = narrowed_cities.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
        
#Create DataFrame        
narrowed_cities["Hotel Name"] = hotel_df

#Drop Null Values
narrowed_cities = narrowed_cities.dropna(how='any')

#Show DataFrame
narrowed_cities.head(10)

,City,Country,Lat,Lng,Date,Max Temp,Humidity,Clouds,Wind Speed,Hotel Name
0,Seydi,TM,39.48,62.91,1601716965,75.87,17,0,5.19,Seydi
1,Atar,MR,20.52,-13.05,1601717042,78.80,44,0,16.11,Atar
2,Cabo San Lucas,MX,22.89,-109.91,1601717023,78.01,75,0,1.99,Cabo San Lucas
3,Qarqīn,AF,37.42,66.04,1601717058,79.81,15,0,3.74,Qarqin
4,Akşehir,TR,38.36,31.42,1601717063,70.68,25,0,3.58,Akşehir
5,Guerrero Negro,MX,27.98,-114.06,1601717109,73.78,66,0,6.02,Guerrero Negro
6,Coahuayana Viejo,MX,18.73,-103.68,1601717143,72.82,57,0,5.41,Coahuayana de Hidalgo
7,Erzin,TR,36.96,36.20,1601717193,79.00,67,0,5.66,Erzin
8,São João da Barra,BR,-21.64,-41.05,1601717256,73.40,94,0,11.41,Grussaí
9,San Borja,BO,-14.82,-66.85,1601717263,75.43,63,0,1.70,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_cities.iterrows()]
locations = narrowed_cities[["Lat", "Lng"]]

In [11]:
#Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
#Save Map as PNG
plt.savefig("Images/Heat Map with Hotels.png")

<Figure size 432x288 with 0 Axes>